In [1]:
#from google.colab import drive
#drive.mount('/content/gdrive')

In [2]:
%matplotlib inline
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Hyperparameters
batch_size = 64
input_size = 784
hidden_size_1 = 512 # 4096
hidden_size_2 = 256 # 2048
hidden_size_3 = 128 # 1024
num_classes = 10

# Learning rate
lr = 0.0001
# Number of training epochs
num_epochs = 20
# Device configuration
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Path to dataset
dataset_root_path = '/scratch2/datasets/FashionMNIST/'
#dataset_root_path = '/content/gdrive/My Drive/FashionMNIST/'

In [4]:
trainset = torchvision.datasets.FashionMNIST(root=dataset_root_path, train=True,\
                                             download=True, transform=transforms.ToTensor())
testset = torchvision.datasets.FashionMNIST(root=dataset_root_path, train=False,\
                                            download=True, transform=transforms.ToTensor())

PermissionError: [Errno 13] Permission denied: '/scratch2'

In [ ]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

In [ ]:
print(trainloader.dataset.train_data.shape)
print(trainloader.dataset.train_labels.shape)

In [ ]:
# Define the class strings for visualization -- The description is in
# https://github.com/zalandoresearch/fashion-mnist#Labels
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle-boot')

Let us show some of the training images, for fun

---



In [ ]:
# Function to show an image
def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1,2,0)), interpolation='nearest')
    plt.axis('off')
    plt.show()

In [ ]:
# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()
print(images.shape)
print(labels.shape)

# show images
imshow(torchvision.utils.make_grid(images[:4],nrow=4))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [ ]:
# Fully connected neural network with three hidden layer
class NeuralNetThreeLayer(nn.Module):
    def __init__(self, input_size, hidden_size_1, hidden_size_2, hidden_size_3, num_classes):
        super(NeuralNetThreeLayer, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size_1)
        self.fc2 = nn.Linear(hidden_size_1, hidden_size_2)
        self.fc3 = nn.Linear(hidden_size_2, hidden_size_3)
        self.fc4 = nn.Linear(hidden_size_3, num_classes)
        
        self.relu = nn.ReLU()
        
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.fc4(x)
        return x

net = NeuralNetThreeLayer(input_size, hidden_size_1, hidden_size_2, hidden_size_3, num_classes)
net = net.to(device)

Define a Loss function and optimizer

---



In [ ]:
criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)
optimizer = optim.Adam(net.parameters(), lr=lr)

In [ ]:
# Helper function to test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
def eval(testloader, epoch):
    # print('here')
    with torch.no_grad():
        running_loss_test = 0.0
        test_correct = 0
        test_total = 0
        test_i = 0
        for test_images, test_labels in testloader:
            # get the inputs
            test_images = test_images.reshape(-1, 28*28).to(device)
            test_labels = test_labels.to(device)
            test_outputs = net(test_images)
            test_loss = criterion(test_outputs, test_labels)
            _, test_predicted = torch.max(test_outputs.data, 1)
            
            running_loss_test += test_loss.item()
            test_total += test_labels.size(0)
            test_correct += (test_predicted == test_labels).sum().item()
            test_i += 1

            if test_total >= testloader.dataset.test_data.shape[0]-batch_size:
                print('[{:d}], test accuracy: {:4.2f} %, test_loss {:.5f}'.format(epoch+1, \
                                                100*test_correct/test_total, running_loss_test/(test_i+1)))
                break
        return 100*test_correct/test_total, running_loss_test/(test_i+1)

In [ ]:
# Eval on training data. This is to be run with initial model only
# In eval phase, we don't need to compute gradients (for memory efficiency)
def eval_initial_model(trainloader):
    with torch.no_grad():
        running_loss_train = 0.0
        train_correct = 0
        train_total = 0
        train_i = 0
        for train_images, train_labels in trainloader:
            # get the inputs
            train_images = train_images.reshape(-1, 28*28).to(device)
            train_labels = train_labels.to(device)
            train_outputs = net(train_images)
            train_loss = criterion(train_outputs, train_labels)
            _, train_predicted = torch.max(train_outputs.data, 1)
            
            running_loss_train += train_loss.item()
            train_total += train_labels.size(0)
            train_correct += (train_predicted == train_labels).sum().item()
            train_i += 1
        
        print('[0], train accuracy: {:4.2f} %, train_loss {:.5f}'.format(100*train_correct/train_total, \
                                                                        running_loss_train/(train_i+1)))
        return 100*train_correct/train_total, running_loss_train/(train_i+1)

In [ ]:
test_acc = [0.0]*(num_epochs+1)
test_loss = [0.0]*(num_epochs+1)
train_acc = [0.0]*(num_epochs+1)
train_loss = [0.0]*(num_epochs+1)
lst_counter = 0

train_acc[lst_counter], train_loss[lst_counter] = eval_initial_model(trainloader)
test_acc[lst_counter], test_loss[lst_counter] = eval(testloader, 0)
lst_counter += 1

for epoch in range(num_epochs):  # loop over the dataset multiple times

    running_loss_training = 0.0
    correct = 0
    total = 0
    i = 0
    # for i, data in enumerate(trainloader, 0):
    for inputs, labels in trainloader:
        # get the inputs
        # inputs, labels = data
        inputs = inputs.reshape(-1, 28*28).to(device); labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        training_loss = criterion(outputs, labels)
        _, predicted = torch.max(outputs.data, 1)
        
        training_loss.backward()
        optimizer.step()

        # print statistics
        running_loss_training += training_loss.item()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        i += 1
        if total >= trainloader.dataset.train_data.shape[0]: # or (epoch == 0 and i == 0):
            print('[{:d}] train accuracy: {:4.2f} % train loss: {:.5f}'.format(epoch+1, \
                                                            100*correct/total, running_loss_training/(i+1)))
            train_acc[lst_counter] = 100*correct/total
            train_loss[lst_counter] = running_loss_training/(i+1)
            
            running_loss_training = 0.0
            
            test_acc[lst_counter], test_loss[lst_counter] = eval(testloader, epoch)
            lst_counter += 1

print('Finished Training')

In [ ]:
# Plotting train and test loss vs Epochs
fig1 = plt.figure(1)        
plt.plot(range(num_epochs+1),train_loss,'r-',label='train_loss')        
plt.plot(range(num_epochs+1),test_loss,'g-',label='test_loss')
plt.legend(loc='upper right')
plt.xlabel('Epochs')
plt.ylabel('Losses')
plt.show()

In [ ]:
fig2 = plt.figure(1)        
plt.plot(range(1, num_epochs+1),train_loss[1:],'r-',label='train_loss')        
plt.plot(range(1, num_epochs+1),test_loss[1:],'g-',label='test_loss')
plt.legend(loc='upper right')
plt.xlabel('Epochs')
plt.ylabel('Losses')
plt.show()

In [ ]:
# Plotting train and test accuracy vs Epochs
fig3 = plt.figure(1)        
plt.plot(range(num_epochs+1),train_acc,'r-',label='train_acc')        
plt.plot(range(num_epochs+1),test_acc,'g-',label='test_acc')
plt.legend(loc='lower right')
plt.xlabel('Epochs')
plt.ylabel('Accuracies')
plt.show()

In [ ]:
dataiter = iter(testloader)
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images[:4], nrow=4))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

# Get the predicted outputs
images = images.reshape(-1, 28*28).to(device)
labels = labels.to(device)
outputs = net(images)
outputs_softmax = F.softmax(outputs, dim=1)
probabilities, predicted = torch.max(outputs_softmax, 1)
probabilities_np = probabilities.cpu().data.numpy()
print('Predicted: ', ' '.join('%5s (%.2f)' % (classes[predicted[j]], probabilities[j]) for j in range(4)))